In [1]:
import os
import seaborn as sns
path = '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Analysis/Figures'
import sys
sys.path.insert(1, '/Users/connormcdonald/Desktop/Masters/MIT807/Gartner Repository/Data Collection')
from configs import *
from scipy import stats
import numpy as np
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.axisartist.axislines import SubplotZero
from pylab import text
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

from sqlalchemy import create_engine

engine = create_engine('mysql+mysqlconnector://'+user+':'+passwd+'@'+ip+':3306/'+schema1)

In [2]:
df = pd.read_csv('/Users/connormcdonald/Desktop/Masters/MIT807/Data/twitterMAU.csv')
df['period']  = pd.to_datetime(df['period'])

In [3]:
df1 = df.iloc[:38,:]

In [4]:
import matplotlib.dates as mdates


degree = 3
y = np.array(df1['mau'].to_list())
X = np.array(mdates.date2num(df1['period']))
date_index = []
idx = 0
for i in X:
    date_index.append(idx)
    idx += 1

z = np.polyfit(date_index, y, degree)
f = np.poly1d(z)

df_trend = pd.DataFrame(columns=['X', 'y','d'])
df_trend['X'] = date_index
df_trend['y'] = y
df_trend['d'] = X


z = np.polyfit(df_trend.d, df_trend.y, degree)
model = np.poly1d(z)
results = smf.ols(formula='y ~ model(X)', data=df_trend).fit()

# print(results.summary().as_latex())

In [5]:
X = mdates.date2num(df['period'])

In [6]:
x_fit = mdates.date2num(df['period'])
y_fit = [model(_x) for _x in x_fit]

In [7]:
def dformat(d):
    return d.strftime('%Y-%m')

def yformat(d):
    return d.strftime('%Y')

In [8]:
df['y_fit'] = y_fit
df['combined_mau'] = np.where(df["mau"].isnull(), df["y_fit"], df["mau"] )*1000000
df['period_formatted'] = df['period'].apply(dformat)

In [9]:
stmt = '''SELECT date, retweet_count FROM social.five_g_only;'''
df2 = pd.read_sql(stmt, con=engine)


In [10]:
df2.shape

(4840197, 2)

In [11]:

df2 = df2[(np.abs(stats.zscore(df2.retweet_count)) < 3)]

In [12]:
df2.head()

,date,retweet_count
0,2010-01-30,0
1,2010-01-30,0
2,2010-01-30,0
3,2010-01-30,0
4,2010-01-30,0


In [13]:
def rep(str):
    strValue = str[:-3]
    return strValue

In [14]:
df2['date'] = df2['date'].apply(rep)

In [15]:
df2['date'] = pd.to_datetime(df2['date'])
df2 = df2.groupby(df2.date)['retweet_count'].sum()

In [16]:
df2 = pd.DataFrame(df2)

In [17]:
df2.reset_index(inplace=True)
df2 = df2.rename(columns = {'index':'date'})

In [18]:
df2

,date,retweet_count
0,2010-01-01,128
1,2010-02-01,273
2,2010-03-01,253
3,2010-04-01,547
4,2010-05-01,276
...,...,...
105,2018-10-01,21498611
106,2018-11-01,119696155
107,2018-12-01,11790267
108,2019-01-01,48133214


In [19]:
df2["date"] = df2["date"].apply(dformat)

In [20]:
df3 = pd.merge(df, df2, left_on='period_formatted', right_on='date', how='inner')

In [21]:
df3['retweets_p_user'] = df3['retweet_count']/df3['combined_mau']

In [23]:
df3['date'] = pd.to_datetime(df3['date'])
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.rcParams["figure.figsize"] = [8, 4]
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.dpi'] = 300


#define colors to use in chart
color_map = ['#FF9A00']

#create area chart
plt.stackplot(df3['date'], df3['retweets_p_user'],
              labels=['Retweets'],
              colors=color_map,
              alpha=0.6)

#add legend
plt.legend(loc='upper left')

#add axis labels
plt.xlabel('Time')
plt.ylabel('Monthly Retweets Per Active User')
plt.show()
plt.savefig(os.path.join(path, '5g_normalised_area3.pdf'), format='pdf',bbox_inches='tight',pad_inches = 0)
plt.close()

<ipython-input-23-a067d8718ca9>:25: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
